# Delta Table 예시

In [1]:
from delta.tables import *
from pyspark.sql.functions import *
import pyspark
from delta import *

In [2]:
builder = pyspark.sql.SparkSession.builder.appName("MyApp2") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()
spark.sparkContext.setLogLevel("WARN") # ALL, DEBUG, ERROR, FATAL, INFO, OFF, TRACE, WARN

data_path  = "/home/donghee/work/spark-3.5.0-bin-hadoop3/data/delta-table-quickstart"

your 131072x1 screen size is bogus. expect trouble


24/02/02 00:07:12 WARN Utils: Your hostname, DESKTOP-JJQA3IT resolves to a loopback address: 127.0.1.1; using 172.25.190.30 instead (on interface eth0)
24/02/02 00:07:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/donghee/miniconda3/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/donghee/.ivy2/cache
The jars for the packages stored in: /home/donghee/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-3c37efb2-4502-4bbb-af1a-98455b8e8015;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.2.0 in central
	found io.delta#delta-storage;2.2.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
:: resolution report :: resolve 132ms :: artifacts dl 3ms
	:: modules in use:
	io.delta#delta-core_2.12;2.2.0 from central in [default]
	io.delta#delta-storage;2.2.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |  

24/02/02 00:07:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
print("################ origin data ################")
deltaTable = DeltaTable.forPath(spark, data_path)
deltaTable.toDF().show()

################ origin data ################
24/02/02 00:07:27 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---+
| id|
+---+
| 37|
| 35|
| 39|
| 33|
| 36|
| 38|
| 31|
| 32|
| 34|
| 30|
+---+



In [4]:
print("################ update data ################")
# Update every even value by adding 100 to it
deltaTable.update(
  condition = expr("id % 2 == 0"),
  set = { "id": expr("id + 100") })
deltaTable.toDF().show()

################ update data ################
+---+
| id|
+---+
| 37|
| 35|
| 39|
| 33|
|138|
|136|
|130|
|132|
| 31|
|134|
+---+



In [5]:
print("################ delete data ################")
# Delete every even value
deltaTable.delete(condition = expr("id % 2 == 0"))
deltaTable.toDF().show()

################ delete data ################
+---+
| id|
+---+
| 37|
| 35|
| 39|
| 33|
| 31|
+---+



In [6]:
# Upsert (merge) new data
newData = spark.range(0, 20)

In [7]:
print("################ update insert data ################")
deltaTable.alias("oldData") \
  .merge(
    newData.alias("newData"),
    "oldData.id = newData.id") \
  .whenMatchedUpdate(set = { "id": col("newData.id") }) \
  .whenNotMatchedInsert(values = { "id": col("newData.id") }) \
  .execute()

deltaTable.toDF().show()

################ update insert data ################
24/02/02 00:09:48 WARN HintErrorLogger: Hint (strategy=broadcast) is not supported in the query: build left for full outer join.
24/02/02 00:09:48 WARN HintErrorLogger: Hint (strategy=broadcast) is not supported in the query: build left for full outer join.
+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  7|
|  6|
|  9|
|  8|
| 10|
| 12|
| 11|
| 13|
| 14|
| 15|
| 17|
| 16|
| 19|
| 18|
+---+
only showing top 20 rows

